## Deliverable 2. Create a Customer Travel Destinations Map.

In [44]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [45]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Boralday,43.3578,76.8594,53.60,54,40,6.71,KZ,2021-04-15 20:50:03
1,1,Bull Savanna,17.8868,-77.5902,80.96,55,18,9.62,JM,2021-04-15 20:50:03
2,2,Ushuaia,-54.8000,-68.3000,50.00,50,40,6.91,AR,2021-04-15 20:44:49
3,3,San Patricio,28.0170,-97.5169,73.40,94,90,13.80,US,2021-04-15 20:46:10
4,4,Ystad,55.4297,13.8204,37.99,75,0,2.30,SE,2021-04-15 20:50:04


In [38]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [39]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temps_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]

In [40]:
# 4a. Determine if there are any empty rows.
temps_df.count()

City_ID       223
City          223
Lat           223
Lng           223
Max Temp      223
Humidity      223
Cloudiness    223
Wind Speed    223
Country       220
Date          223
dtype: int64

In [41]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = temps_df.dropna()
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Bull Savanna,17.8868,-77.5902,80.96,55,18,9.62,JM,2021-04-15 20:50:03
8,8,Bambous Virieux,-20.3428,57.7575,78.01,94,75,5.75,MU,2021-04-15 20:45:18
12,12,Jensen Beach,27.2545,-80.2298,89.01,29,20,9.22,US,2021-04-15 20:50:06
14,14,Vaini,-21.2000,-175.2000,78.80,78,40,8.05,TO,2021-04-15 20:45:14
16,16,Avarua,-21.2078,-159.7750,82.40,74,17,9.22,CK,2021-04-15 20:45:09
...,...,...,...,...,...,...,...,...,...,...
697,697,Meadow Lake,34.8014,-106.5436,75.99,8,75,19.57,US,2021-04-15 20:53:02
702,702,Porto Novo,6.4965,2.6036,86.00,74,20,12.66,BJ,2021-04-15 20:53:04
705,705,Damaturu,11.7470,11.9608,84.13,5,75,9.24,NG,2021-04-15 20:53:05
708,708,Maceio,-9.6658,-35.7353,78.80,83,20,5.75,BR,2021-04-15 20:50:00


In [42]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

KeyError: "['Current Description'] not in index"

In [43]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

NameError: name 'hotel_df' is not defined

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how = 'any', subset = ['Hotel Name'])

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

In [ ]:
# Save figure
plt.savefig("../Vacation_Search/WeatherPy_vacation_map.png")